In [2]:
!pip install transformers tensorflow

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 32.0 MB/s 
     |████████████████████████████████| 6.5 MB 20.5 MB/s 
     |████████████████████████████████| 596 kB 30.2 MB/s 
     |████████████████████████████████| 67 kB 3.9 MB/s 
     |████████████████████████████████| 462 kB 33.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification
import tensorflow as tf
import csv


In [7]:
text = []
labels = []

with open('/content/dataset.csv') as fp:
  reader = csv.reader(fp,delimiter=',')
  for row in reader:
    text.append(row[2])
    labels.append(row[3])
text = text[1:]
labels = labels[1:]
for i,val in enumerate(labels):
  labels[i] = int(labels[i])
tot_size = len(text)
train_ratio = 0.9
train_len=int(train_ratio*tot_size)
train_text = text[:train_len]
train_labels = labels[:train_len]

val_text = text[train_len:]
val_labels = labels[train_len:]




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
training_size = 20000

with open("/content/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

train_text = sentences[0:training_size]
val_text = sentences[training_size:]
train_labels = labels[0:training_size]
val_labels = labels[training_size:]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

In [9]:
train_encodings = tokenizer(train_text,truncation=True,padding=True)

val_encodings = tokenizer(val_text,truncation=True,padding=True)

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [11]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

Downloading:   0%|          | 0.00/60.1M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFAlbertForSequenceClassification.

Some layers of TFAlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])
model.fit(train_dataset.shuffle(len(train_text)).batch(16),
          epochs=3,
          batch_size=16)

Epoch 1/3
1503/1503 [==============================] - 793s 512ms/step - loss: 0.2845 - accuracy: 0.8806
Epoch 2/3
1503/1503 [==============================] - 769s 511ms/step - loss: 0.1591 - accuracy: 0.9417
Epoch 3/3
1503/1503 [==============================] - 769s 511ms/step - loss: 0.0963 - accuracy: 0.9666


In [14]:
model.evaluate(val_dataset.shuffle(len(val_text)).batch(16),
               return_dict=True,
               batch_size=16)

167/167 [==============================] - 22s 112ms/step - loss: 0.3626 - accuracy: 0.8824


{'accuracy': 0.8824410438537598, 'loss': 0.3626142144203186}

In [15]:
model.save_pretrained('/content/drive/MyDrive/BTP/albert-base-v2')